**Warning: Offensive language**

In [1]:
from ml_pipeline import experiment

/home/rutger/miniconda3/envs/pynlp/lib/python3.7/site-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.metrics.classification module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)
2022-09-25 11:39:40.539963: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory
2022-09-25 11:39:40.539993: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
pipe = experiment.run(
    task_name='vua_format',
    data_dir='../pynlp/data/dataset1/',
    pipeline_name='naive_bayes_counts_lex',
    print_predictions=False
)

>> Running vua_format experiment
>> Loading data...
>> retrieving train/data instances...
>> training pipeline naive_bayes_counts_lex
>> testing...
>> evaluation...
              precision    recall  f1-score   support

         NOT       0.80      0.62      0.70      1972
         OFF       0.62      0.81      0.70      1539

    accuracy                           0.70      3511
   macro avg       0.71      0.71      0.70      3511
weighted avg       0.72      0.70      0.70      3511



In [3]:
import pandas as pd

In [4]:
df_gibert_test = pd.read_csv('../pynlp/data/gibert/testData.csv', delimiter='\t')

In [5]:
pipe

Pipeline(steps=[('prep',
                 <ml_pipeline.preprocessing.Preprocessor object at 0x7f36c9231c90>),
                ('frm', CountVectorizer()), ('clf', MultinomialNB())])

In [6]:
prep = pipe['prep']
frm = pipe['frm']
clf = pipe['clf']

In [7]:
x = df_gibert_test['Text']

In [8]:
res = prep.transform(x)

print('Before processing:')
print(x[1])
print('\n')
print('After processing:')
print(res[1])

Before processing:
I know exactly what you 're saying , I just hate to see those worthless apes benefit from the great things that whites do .


After processing:
I know exactly what you ' re saying , I just hate to see those worthless hate apes benefit from the great things that whites do .


With a lexicon `hate` is prepended to any token that is found in the lexicon. In this case `apes` is in the lexicon:
```
Entry;Pos;Label
...
apes;anypos;hate
...
```
hence \
```...hate to see those worthless apes benefit...``` \
is transformed to  \
```...hate to see those worthless hate apes benefit...```

In [9]:
x[1]

"I know exactly what you 're saying , I just hate to see those worthless apes benefit from the great things that whites do ."

In [10]:
pipe.predict([x[1]])

array(['OFF'], dtype='<U3')

In [11]:
d_train = pd.read_csv('../pynlp/data/dataset1/trainData.csv', delimiter='\t')

In [12]:
manual_input = [
    'hate'
]
pred = pipe.predict(manual_input)
pd.DataFrame({
    'text': manual_input,
    'prediction': pred
})

,text,prediction
0,hate,OFF


As words in our lexicon are prepended with `hate`, any text containing the word `hate` should be much more likely to be predicted as offensive.

In [13]:
d_train[d_train['Text'].str.contains('hate')].groupby('Label')[['Id']].count()

,Id
Label,
NOT,167
OFF,289


**Differentiate between token `hate` added with lexicon preprocessor (e.g. add token `[HATE]`), vs. the word `hate`?**

## `I hate apples` vs. `I hate oranges`

In [14]:
manual_input = [
    'I hate apples',
    'I hate oranges'
]
pred = pipe.predict(manual_input)

pd.DataFrame({
    'text': manual_input,
    'prediction': pred
})

,text,prediction
0,I hate apples,NOT
1,I hate oranges,OFF


In [15]:
stats1 = d_train[d_train['Text'].str.contains('apples')].groupby('Label')[['Id']].count()
stats2 = d_train[d_train['Text'].str.contains('oranges')].groupby('Label')[['Id']].count()
stats1.join(stats2, lsuffix=' Apples', rsuffix=' Oranges').T

Label,NOT,OFF
Id Apples,5,1
Id Oranges,1,1


Since there is less training data available containing the word oranges, the model seems to rely on the word `hate` to predict it as `offensive`. For `apples`, more data is available that is not offensive.

## `I hate apple` vs. `I hate orange`

In [16]:
manual_input = [
    'I hate apple',
    'I hate orange'
]
pred = pipe.predict(manual_input)

pd.DataFrame({
    'text': manual_input,
    'prediction': pred
})

,text,prediction
0,I hate apple,OFF
1,I hate orange,OFF


In [17]:
stats1 = d_train[d_train['Text'].str.contains('apple')].groupby('Label')[['Id']].count()
stats2 = d_train[d_train['Text'].str.contains('orange')].groupby('Label')[['Id']].count()
stats1.join(stats2, lsuffix=' Apple', rsuffix=' Orange').T

Label,NOT,OFF
Id Apple,8,9
Id Orange,5,5


Plural `apples` is predicted as not offensive, whereas singular `apple` is predicted as offensive. Probably because of more uncertainty.

$$\mathbf{P}(\text{Hate }|\text{ text contains 'apples'})=0.2$$
$$\mathbf{P}(\text{Hate }|\text{ text contains 'apple'})=0.53$$

In [18]:
import numpy as np 

np.array_equal(
    frm.transform(prep.transform(['I hate apple'])).todense(),
    frm.transform(prep.transform(['I hate apple'])).todense()
)

True

In [19]:
np.array_equal(
    frm.transform(prep.transform(['I hate apple'])).todense(),
    frm.transform(prep.transform(['I hate apples'])).todense()
)

False

In [20]:
frm.transform(prep.transform(['apple'])).todok().items()

dict_items([((0, 2596), 1)])

In [21]:
frm.transform(prep.transform(['apples'])).todok().items()

dict_items([((0, 2598), 1)])

2 tokens in corpus for `apple` and `apples`. **Maybe use subword tokenizer?**